# Interactive use

This notebook includes an interactive demonstration.  First, we set up the environment, by installing then loading the dependencies:

In [1]:
# for jupyter lite (if you are not working locally, run this cell to install the dependencies)
%pip install svg.py numpy matplotlib pandas ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [1]:
from lmunplugged import Bin,Table, Ball, Sticky, Doc, TrainDemo
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

The next cell will create an interactive interface you can use to experiment with the training process. It does not yet handle the end document well, it will cycle back to the beginning if you press train an additional time after it adds the white ball. 

In [ ]:
# This puts custom CSS in so that the words in the "author" space are bigger
display(HTML("""
<style>
.jupyter-widget-tag { 
    width: 70px;
    height: 50px;
}
</style>
"""))


initital_doc = ['green','blue','purple','blue','pink','blue','white']
bin_colors = ['purple','blue','green','pink']
allowed_colors = bin_colors + ['white']
color_tags = widgets.ColorsInput(
    value=initital_doc,
    allowed_tags=allowed_colors,
    # allow_duplicates=False
)

my_doc = Doc.from_list(color_tags.value,max_width_words=5)
to_train = Table.from_list(bin_colors)

my_demo = TrainDemo(to_train, my_doc)

html_output_demo= widgets.Output()
with html_output_demo:
    display(my_demo)

def set_demo_button(b):
    my_demo.doc.reset_words(color_tags.value)
    my_demo.reset_training()
    with html_output_demo:
        html_output_demo.clear_output(wait=True)
        display(my_demo)


def train_step(b):    
    with html_output_demo:
        html_output_demo.clear_output(wait=True)
        display(my_demo.train_next())



demo_button = widgets.Button(
    description="load doc",
    button_style='info', 
    layout=widgets.Layout(width='100px', height='30px')
)

step_button = widgets.Button(
    description="step",
    button_style='success', 
    layout=widgets.Layout(width='100px', height='30px')
)

demo_button.on_click(set_demo_button)
step_button.on_click(train_step)
document = widgets.VBox([
    widgets.HTML("<h2>Interactive Training Demo</h2>"),
    widgets.HTML("<p>Mimic training a tiny language model from a" \
    " document comprised of sticky notes. Use the x to remove or " \
    "click to the right of the last colored rectangle and choose "
    "a color to add one. Drag and drop them re-order. "
    "Be sure the last one is white.</p>"),
    color_tags,
    widgets.HTML("<p>When you want to train, click the "
                 "'load document' button below to load the your "
                 "document, then click 'step' to train the model.</p>")
])
demo_ctrl = widgets.HBox([ demo_button,step_button])
layout_train = widgets.VBox([document,demo_ctrl,html_output_demo])

Finally, the following cell actually **shows** the interactive training demo.  You can create your own training file by edi

In [79]:
display(layout_train)

## Sampling

the next cell defines an interactive sampling and prompting setup

In [ ]:
prompter_toggle = widgets.ToggleButtons(
    options=bin_colors,
    description='Prompt:',
    disabled=False,
    button_style='', 
    # 'success', 'info', 'warning', 'danger' or ''
#     icons=['check'] * 3
)
button_list = [widgets.Button(description='        ',
    style={'button_color': Sticky(cur_color).color}, 
    layout=widgets.Layout(width='70px', height='50px'),
    )  for cur_color in bin_colors]

# [bt.on_click(lambda b: prompt(b,color=cur_color)) for bt, cur_color in zip(button_list, bin_colors)]
button_list[0].on_click(lambda b: prompt(b,color=bin_colors[0]))
button_list[1].on_click(lambda b: prompt(b,color=bin_colors[1]))
button_list[2].on_click(lambda b: prompt(b,color=bin_colors[2]))
button_list[3].on_click(lambda b: prompt(b,color=bin_colors[3]))
# btfx = [lambda b: prompt(b,color=cur_color) 
#            for cur_color in bin_colors]

# [bt.on_click(fx) 
#            for bt, fx in zip(button_list, btfx)]

# button_list[2].style.button_color = 'lightblue'  # Highlight the blue button
prompter_buttons = widgets.HBox(button_list)

prompter_color = widgets.ColorsInput(
    value=[],
    allowed_tags=bin_colors,
    # allow_duplicates=False
)

doc = Doc(["white"],max_width_words=5)
html_output_sampler= widgets.Output()
with html_output_sampler:
    display(doc)

load_button = widgets.Button(
    description="select prompt",
    button_style='info', 
    layout=widgets.Layout(width='100px', height='30px')
)
def prompt(b, color):
    with html_output_sampler:
        
    # my_demo.doc.reset_words(color_tags.value)
        html_output_sampler.clear_output(wait=True)
        doc.reset_words([color])
        display(doc)
        # display(to_train.sample_doc(prompter.value))

sample_button = widgets.Button(
    description="sample",
    button_style='info', 
    layout=widgets.Layout(width='100px', height='30px',background_color='lightblue'),
)


def sample(b):
    with html_output_sampler:
        html_output_sampler.clear_output(wait=True)
        # doc.reset_words(prompter_color.value)
        display(to_train.sample_doc(doc.get_word(0).name))



load_button.on_click(prompt)
sample_button.on_click(sample)
# to_train.sample_doc('green')

controls = widgets.HBox([prompter_buttons,sample_button])

sampler = widgets.VBox([
    widgets.HTML("<h2>Interactive Sampling Demo</h2>"),
    widgets.HTML("<p>click a sticky to prompt in that color then " \
    " click sample to get a document.</p>"),controls,html_output_sampler])

The next cell displays the sampling demo:

In [81]:
display(sampler)